In [ ]:
import json
import sys

from modules.limu_model import limu_model4pretrain
from modules.pretrain_hyperparameters import LIMU_Pretrain_Hyperparameters
from utils.load_data_from_file import load_mixed_data, prepare_mixed_data_loader, load_one_out_data, \
    prepare_one_out_data_loader, limu_prepare_mixed_data_loader, limu_prepare_one_out_data_loader
from utils.pretrain import pretrain_limu_model


def main():
    # Load the config from JSON file first
    with open("utils/config.json", "r") as file:
        config = json.load(file)
    print(config)

    if config["general"]["test_mode"] == "Mixed":
        data, labels, encoder = load_mixed_data(window_size=config["general"]["window_size"],
                                                overlap=config["general"]["overlap"])

        num_classes = len(encoder.classes_)
        feat_dim = data[0].shape[1]
        labels_dim = labels.shape
        print(f"The shape of data is {data.shape}, the feat_dim is {feat_dim}, the labels_dim is {labels_dim}")

        eyegaze_data_loader = (limu_prepare_mixed_data_loader
                               (config, data, labels, batch_size=config["general"]["batch_size"],
                                max_len=config["general"]["window_size"]))

    elif config["general"]["test_mode"] == "One_out":
        train_data, train_labels, test_data, test_labels, encoder = (load_one_out_data
                                                                     (window_size=config["general"]["window_size"],
                                                                      overlap=config["general"]["overlap"]))

        num_classes = len(encoder.classes_)
        feat_dim = train_data[0].shape[1]
        print(f"The number of classes is {num_classes}, the feat_dim is {feat_dim}")

        eyegaze_data_loader = (limu_prepare_one_out_data_loader
                               (config, train_data, train_labels, test_data, test_labels,
                                batch_size=config["general"]["batch_size"],
                                max_len=config["general"]["window_size"]))
    else:
        print("Either Mixed / One_out")
        sys.exit()

    hyperparameters = LIMU_Pretrain_Hyperparameters(config)
    model = limu_model4pretrain(config, feat_dim)
    loss = hyperparameters.loss
    optimizer = hyperparameters.optimizer(model.parameters(), hyperparameters.lr, weight_decay=hyperparameters.weight_decay)

    pretrain_limu_model(model, loss, optimizer, eyegaze_data_loader[0], config)


if __name__ == "__main__":
    main()


{'general': {'test_mode': 'One_out', 'window_size': 150, 'overlap': 0.8, 'pretrain_model': None, 'batch_size': 128, 'freeze': False}, 'kdd_pretrain': {'epoch': 3200, 'lr': 0.0001, 'optimizer': 'RAdam', 'weight_decay': None, 'harden': False}, 'limu_pretrain': {'epoch': 3200, 'lr': 0.0001, 'optimizer': 'Adam', 'weight_decay': None, 'harden': False}, 'limu_mask': {'mask_ratio': 0.15, 'mask_alpha': 6, 'max_gram': 10, 'mask_prob': 0.8, 'replace_prob': 0.0}, 'kdd_model': {'d_hidden': 128, 'd_ff': 256, 'n_heads': 16, 'n_layers': 3, 'dropout': 0.1, 'pos_encoding': 'learnable', 'activation': 'gelu', 'norm': 'BatchNorm', 'projection': 'linear'}, 'limu_model': {'d_hidden': 72, 'd_ff': 144, 'n_heads': 4, 'n_layers': 4, 'emb_norm': False}}
Class: BROWSE -> Encoded Value: 0
Class: PLAY -> Encoded Value: 1
Class: READ -> Encoded Value: 2
Class: SEARCH -> Encoded Value: 3
Class: WATCH -> Encoded Value: 4
Class: WRITE -> Encoded Value: 5
The number of classes is 6, the feat_dim is 2
Model:
LIMUBertMode